In [1]:
import geemap
import ee
import numpy as np
import folium
import os
import pprint
import time
geemap.ee_initialize()

In [2]:
maxReducer = ee.Reducer.max()
minReducer = ee.Reducer.min()

In [3]:
# def add_ee_layer
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [4]:
#load data
def load_data(geeID, data_type, *bandNames): 
    #image: data_type=0; ImageCollection: data_type=1; FeatureCollection: data_type
    #bandNames: rename bands for Image or ImageCollection; i.e. ['Red', 'Green', 'Blue', 'NIR'] for Planet Scope
    if data_type == 0:
        image = ee.Image(geeID)
        if bandNames != None:
            image = image.rename(bandNames)
        return image
    elif data_type == 1:
        imageCollection = ee.ImageCollection(geeID)
        return imageCollection
    elif data_tpye == 2:
        geometry = ee.FeatureCollection(geeID).geometry()
        return geometry

In [5]:
def show_image_geemap(image, gamma=1):
    Map = geemap.Map()
    Map.centerObject(image)
    maxValue = image.reduceRegion(maxReducer, bestEffort=True).getInfo()['Blue']
    minValue = image.reduceRegion(minReducer, bestEffort=True).getInfo()['Red']
    img_viz_params = {
        'bands': ['Blue', 'Green', 'Red'],
        'min': minValue,
        'max': maxValue,
        'gamma': gamma
    }
    Map.addLayer(image, img_viz_params, 'image')
    return(Map)

In [6]:
def geo_to_featureCollection(*args, labelName='class'):
    # args: geometries
    l = []
    n = len(args)
    for i in args:
        l.append(ee.Feature(i, {labelName: (n-1)}))
    fC = ee.FeatureCollection(l)
    return fC

In [7]:
#Classification
def classification(image, scale, **kwargs):
    # *kwargs
    # Null
    if "simples" in kwargs:
        #NaiveBayes
        #Three land type simple regions: water, plant, sand (other)
        bands = image.bandNames()
        classif = kwargs.pop("simples")
        labels = classif.first().propertyNames().getInfo()[1]
        training = image.select(bands).sampleRegions(**{
            'collection': classif,
            'properties': [labels],
            'scale' : scale
        })
        trained = ee.Classifier.smileNaiveBayes().train(training, labels, bands)
        classified = image.select(bands).classify(trained)
        water_mask = classified.eq(0)
        sand_mask = classified.eq(2)
        return(water_mask, sand_mask)
    elif "ndvi" in kwargs:
        #NDVI and NDWI
        ndvip = kwargs['ndvi']
        #ndwip = kwargs['ndwi']
        ndvi = image.normalizedDifference(['NIR', 'Red']) #NIR, Red
        #ndwi = image.normalizedDifference(['Green', 'NIR']) #Green, NIR
        ndvi1 = ndvi.lt(ndvip).eq(1)
        #ndwi1 = ndwi.gt(ndwip)
        #water_mask = ndwi1.And(ndvi1).eq(1)
        return(ndvi1)

In [8]:
def noise_removal(mask0, scale, bandName = None):
    #remove other water bodies and noises from the water mask
    #if the bandName of the mask is not given, the first band will be considered the mask
    crs = mask0.projection().crs().getInfo()
    if bandName == None:
        bandName = mask0.bandNames().getInfo()[0]
    mask = mask0.focal_min(1, kernelType='square')\
        .reproject(crs = crs, scale = scale)
    mask1 = mask.addBands(mask.eq(0))\
        .reduceConnectedComponents(ee.Reducer.median(), bandName, 1024)\
        .eq(0).unmask()\
        .reproject(crs = crs, scale = scale)
    mask2 = mask1.add(mask).eq(1)
    mask3 = mask2.focal_max(1, kernelType='square')\
        .reproject(crs = crs, scale = scale)
    return mask3

In [9]:
def large_centerbar_removal(mask0, scale, area, bandName = None):
    mask0 = mask0.Not()
    crs = mask0.projection().crs().getInfo()
    if bandName == None:
        bandName = mask0.bandNames().getInfo()[0]
    mask = mask0.focal_min(1, kernelType='square')\
        .reproject(crs = crs, scale = scale)
    mask1 = mask.addBands(mask.eq(0))\
        .reduceConnectedComponents(ee.Reducer.median(), bandName, area)\
        .eq(0).unmask()\
        .reproject(crs = crs, scale = scale)
    mask2 = mask1.focal_max(1, kernelType='square')\
        .reproject(crs = crs, scale = scale)
    return mask2

In [10]:
def small_centerbar_removal(mask0, scale, area=100, bandName = None):
    mask = mask0.Not()
    crs = mask.projection().crs().getInfo()
    mask1 = mask.connectedPixelCount(area, False).reproject(crs = crs, scale = scale)
    mask2 = mask1.lt(area).selfMask().unmask()
    mask3 = mask0.Or(mask2)
    return mask3

In [11]:
def close(image):
    image = image.focal_max().focal_min()
    return image

In [12]:
def hitOrMiss(image, se1, se2):
    """perform hitOrMiss transform (adapted from [citation])
    """
    e1 = image.reduceNeighborhood(ee.Reducer.min(), se1)
    e2 = image.Not().reduceNeighborhood(ee.Reducer.min(), se2)
    return e1.And(e2)

In [13]:
def splitKernel(kernel, value):
    """recalculate the kernel according to the given foreground value
    """
    kernel = np.array(kernel)
    result = kernel
    r = 0
    while (r < kernel.shape[0]):
        c = 0
        while (c < kernel.shape[1]):
            if kernel[r][c] == value:
                result[r][c] = 1
            else:
                result[r][c] = 0
            c = c + 1
        r = r + 1
    return result.tolist()

In [14]:
def Skeletonize(image, iterations, method):
    """perform skeletonization
    """

    se1w = [[2, 2, 2], [0, 1, 0], [1, 1, 1]]

    if (method == 2):
        se1w = [[2, 2, 2], [0, 1, 0], [0, 1, 0]]

    se11 = ee.Kernel.fixed(3, 3, splitKernel(se1w, 1))
    se12 = ee.Kernel.fixed(3, 3, splitKernel(se1w, 2))

    se2w = [[2, 2, 0], [2, 1, 1], [0, 1, 0]]

    if (method == 2):
        se2w = [[2, 2, 0], [2, 1, 1], [0, 1, 1]]

    se21 = ee.Kernel.fixed(3, 3, splitKernel(se2w, 1))
    se22 = ee.Kernel.fixed(3, 3, splitKernel(se2w, 2))

    result = image

    i = 0
    while (i < iterations):
        j = 0
        while (j < 4):
              result = result.subtract(hitOrMiss(result, se11, se12))
              se11 = se11.rotate(1)
              se12 = se12.rotate(1)
              result = result.subtract(hitOrMiss(result, se21, se22))
              se21 = se21.rotate(1)
              se22 = se22.rotate(1)
              j = j + 1
        i = i + 1

    return result.rename(['clRaw'])

In [15]:
def CalcDistanceMap(img, neighborhoodSize, scale):
    # // assign each river pixel with the distance (in meter) between itself and the closest non-river pixel
    imgD2 = img.focal_max(1.5, 'circle', 'pixels', 2)
    imgD1 = img.focal_max(1.5, 'circle', 'pixels', 1)
    outline = imgD2.subtract(imgD1)

    dpixel = outline.fastDistanceTransform(neighborhoodSize).sqrt()
    dmeters = dpixel.multiply(scale) #// for a given scale
    DM = dmeters.mask(dpixel.lte(neighborhoodSize).And(imgD2))

    return(DM)

In [16]:
def CalcGradientMap(image, gradMethod, scale):
    ## Calculate the gradient
    if (gradMethod == 1): # GEE .gradient() method
        grad = image.gradient()
        dx = grad.select(['x'])
        dy = grad.select(['y'])
        g = dx.multiply(dx).add(dy.multiply(dy)).sqrt()

    if (gradMethod == 2): # Gena's method
        k_dx = ee.Kernel.fixed(3, 3, [[ 1.0/8, 0.0, -1.0/8], [ 2.0/8, 0.0, -2.0/8], [ 1.0/8,  0.0, -1.0/8]])
        k_dy = ee.Kernel.fixed(3, 3, [[ -1.0/8, -2.0/8, -1.0/8], [ 0.0, 0.0, 0.0], [ 1.0/8, 2.0/8, 1.0/8]])
        dx = image.convolve(k_dx)
        dy = image.convolve(k_dy)
        g = dx.multiply(dx).add(dy.multiply(dy)).divide(scale**2).sqrt()

    if (gradMethod == 3): # RivWidth method
        k_dx = ee.Kernel.fixed(3, 1, [[-0.5, 0.0, 0.5]])
        k_dy = ee.Kernel.fixed(1, 3, [[0.5], [0.0], [-0.5]])
        dx = image.convolve(k_dx)
        dy = image.convolve(k_dy)
        g = dx.multiply(dx).add(dy.multiply(dy)).divide(scale.multiply(scale))

    return(g)

In [17]:
def CalcOnePixelWidthCenterline(img, GM, hGrad):
    # /***
    # calculate the 1px centerline from:
    # 1. fast distance transform of the river banks
    # 2. gradient of the distance transform, mask areas where gradient greater than a threshold hGrad
    # 3. apply skeletonization twice to get a 1px centerline
    # thresholding gradient map inspired by Pavelsky and Smith., 2008
    # ***/

    imgD2 = img.focal_max(1.5, 'circle', 'pixels', 2)
    cl = ee.Image(GM).mask(imgD2).lte(hGrad).And(img)
    # // apply skeletonization twice
    cl1px = Skeletonize(cl, 2, 1)
    return(cl1px)

In [18]:
def ExtractEndpoints(CL1px):
    """calculate end points in the one pixel centerline
    """

    se1w = [[0, 0, 0], [2, 1, 2], [2, 2, 2]]

    se11 = ee.Kernel.fixed(3, 3, splitKernel(se1w, 1))
    se12 = ee.Kernel.fixed(3, 3, splitKernel(se1w, 2))

    result = CL1px

    # // the for loop removes the identified endpoints from the imput image
    i = 0
    while (i<4): # rotate kernels
        result = result.subtract(hitOrMiss(CL1px, se11, se12))
        se11 = se11.rotate(1)
        se12 = se12.rotate(1)
        i = i + 1
    endpoints = CL1px.subtract(result)
    return endpoints

In [19]:
def ExtractCorners(CL1px):
    """calculate corners in the one pixel centerline
    """

    se1w = [[2, 2, 0], [2, 1, 1], [0, 1, 0]]

    se11 = ee.Kernel.fixed(3, 3, splitKernel(se1w, 1))
    se12 = ee.Kernel.fixed(3, 3, splitKernel(se1w, 2))

    result = CL1px
    # // the for loop removes the identified corners from the imput image

    i = 0
    while(i < 4): # rotate kernels

        result = result.subtract(hitOrMiss(result, se11, se12))

        se11 = se11.rotate(1)
        se12 = se12.rotate(1)

        i = i + 1

    cornerPoints = CL1px.subtract(result)
    return cornerPoints

In [20]:
def CleanCenterline(cl1px, maxBranchLengthToRemove, rmCorners):
    """clean the 1px centerline:
	1. remove branches
	2. remove corners to insure 1px width (optional)
    """

    ## find the number of connecting pixels (8-connectivity)
    nearbyPoints = (cl1px.mask(cl1px).reduceNeighborhood(
        reducer = ee.Reducer.count(),
        kernel = ee.Kernel.circle(1.5), #1.5
        skipMasked = True))

	## define ends
    endsByNeighbors = nearbyPoints.lte(2)

	## define joint points
    joints = nearbyPoints.gte(4)

    costMap = (cl1px.mask(cl1px).updateMask(joints.Not()).cumulativeCost(
        source = endsByNeighbors.mask(endsByNeighbors),
        maxDistance = maxBranchLengthToRemove,
        geodeticDistance = True))

    branchMask = costMap.gte(0).unmask(0)
    cl1Cleaned = cl1px.updateMask(branchMask.Not()) # mask short branches;
    ends = ExtractEndpoints(cl1Cleaned)
    cl1Cleaned = cl1Cleaned.updateMask(ends.Not())
    cl1Cleaned = close(cl1Cleaned)

    if (rmCorners):
        corners = ExtractCorners(cl1Cleaned)
        cl1Cleaned = cl1Cleaned.updateMask(corners.Not())

    return (cl1Cleaned)

In [21]:
def CalculateCenterline(imgIn, scale):

    crs = imgIn.projection().crs().getInfo()

    distM = CalcDistanceMap(imgIn, 256, scale).reproject(crs = crs, scale = scale)
    gradM = CalcGradientMap(distM, 2, scale).reproject(crs = crs, scale = scale)
    cl1 = CalcOnePixelWidthCenterline(imgIn, gradM, 0.9).reproject(crs = crs, scale = scale)
    cl1Cleaned1 = CleanCenterline(cl1, 300, True).reproject(crs = crs, scale = scale)
    cl1px = CleanCenterline(cl1Cleaned1, 300, False).reproject(crs = crs, scale = scale)
    return(cl1px)

In [26]:
imgC = load_data('projects/ee-alpha-luoyee1997/assets/lt_ps', 1)

In [28]:
from xml.dom import minidom

In [40]:
def getTOA(path):
    xmldoc = minidom.parse(path)
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
    return coeffs

In [30]:
import glob

In [37]:
xmlpath = glob.glob('/Users/luoyi/Box/private/MSDelta/planet scope/LT/*metadata_clip.xml')

In [45]:
getTOA(xmlpath[0])

{1: 2.39272120581e-05,
 2: 2.59509198827e-05,
 3: 3.12355148955e-05,
 4: 4.9437966273e-05}